# Introduction to Machine Learning

<img src="logo.jpg" style="float: left; width: 15%" />

[CSE204-2018](https://moodle.polytechnique.fr/course/view.php?id=6784) Lab session #04

Jérémie DECOCK

<a href="https://colab.research.google.com/github/jeremiedecock/polytechnique-cse204-2018/blob/master/lab_session_04_correction.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a href="https://mybinder.org/v2/gh/jeremiedecock/polytechnique-cse204-2018/master?filepath=lab_session_04_correction.ipynb"><img align="left" src="https://mybinder.org/badge.svg" alt="Open in Binder" title="Open and Execute in Binder"></a>

## Objectives

In the previous lab session, we have used a **parametric models** to solve **regression problems**.
Today you will continue the exploration of regression methods (both parametric and non-parametric).

- Kernel regression
- Weighted Least Squares
- Local Linear Regression
- Pathological cases
- Regularization with Ridge regression

**Note**: there are some differences in notations with the lecture slides. For instance, parameters are noted $w$ in lectures but they are noted $\theta$ here.

## Imports and tool functions

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing

In [ ]:
def gen_1d_polynomial_regression_samples(n_samples = 15):

    #x = np.random.uniform(low=0., high=10., size=n_samples)
    x = np.random.uniform(low=0., high=1.5, size=n_samples)

    #y = 3. - 2. * x + x ** 2 - x ** 3 + np.random.normal(scale=10., size=x.shape)
    y = np.cos(2. * np.pi * x) + np.random.normal(scale=0.1, size=x.shape)

    df = pd.DataFrame(np.array([x, y]).T, columns=['x', 'y'])

    df = sklearn.utils.shuffle(df).reset_index(drop=True)

    return df

In [ ]:
def plot_1d_regression_samples(dataframe, model=None):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    df = dataframe  # make an alias
    
    ERROR_MSG1 = "The `dataframe` parameter should be a Pandas DataFrame having the following columns: ['x', 'y']"
    assert df.columns.values.tolist() == ['x', 'y'], ERROR_MSG1
    
    if model is not None:
        
        # Compute the model's prediction
        
        x_pred = np.linspace(df.x.min(), df.x.max(), 100).reshape(-1, 1)
        y_pred = model.predict(x_pred)
        
        df_pred = pd.DataFrame(np.array([x_pred.flatten(), y_pred.flatten()]).T, columns=['x', 'y'])
        
        df_pred.plot(x='x', y='y', style='r--', ax=ax)

    # Plot also the training points
    
    df.plot.scatter(x='x', y='y', ax=ax)
    
    delta_y = df.y.max() - df.y.min()
    
    plt.ylim((df.y.min() - 0.15 * delta_y,
              df.y.max() + 0.15 * delta_y))

In [ ]:
def plot_regression_1d(X, y, theta=None, x_min=0, x_max=2):
    assert X.ndim == 2 and X.shape[1] == 2, X.shape
    assert y.ndim == 2 and y.shape[1] == 1, y.shape
    if theta is not None:
        assert theta.ndim == 2 and theta.shape == (2, 1), theta.shape
    
    fig, ax = plt.subplots()
    ax.scatter(X[:,1], y)

    if theta is not None:
        x = np.linspace(x_min, x_max, 50)
        y = theta[0] + theta[1] * x

        ax.plot(x, y, "--r")

## Nadaraya-Watson Kernel Regression

Like k-Nearest Neighbors, *Nadaraya-Watson kernel regression* is a non-parametric model, i.e. decisions are made according to known examples from the *learning set* $\mathcal{D} = \{(y^{(i)}, \boldsymbol{x^{(i)}})\}_{1 \leq i \leq n}$ of $n$ examples and considering a kind of proximity relationship.

With k-Nearest Neighbors, decisions are based only on the closest neighbors and others examples are simply ignored.
On the contrary, with Kernel Regression all examples $(\boldsymbol{x}^{(i)}, y^{(i)})$ from $\mathcal{D}$ are used to predict the label $y$ of any new point $\boldsymbol{x}$: the contribution of $(\boldsymbol{x}^{(i)}, y^{(i)})$ in this prediction is weighted using a *kernel function* $K(\boldsymbol{x}^{(i)}, \boldsymbol{x})$. 

$$
y
= f(\boldsymbol{x})
= \frac{\sum^{n}_{i=1} K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) ~ y^{(i)}}{\sum^{n}_{i=1} K(\boldsymbol{x}^{(j)}, \boldsymbol{x})}
= \sum^{n}_{i=1} y^{(i)} \omega^{(i)}
$$

with $\sum^{n}_{i=1} \omega^{(i)} = 1$

Recall about the notation used here:
- $\boldsymbol{x}^{(i)}$ is the feature (input) vector of the $i^{\text{th}}$ example in $\mathcal{D}$ (and $y^{(i)}$ is its label). Here, $\boldsymbol{x}^{(i)}$ is not the $i^{\text{th}}$ power of $\boldsymbol{x}$ (we will write $\boldsymbol{x}^{(i)2}$ for the square of $\boldsymbol{x}^{(i)}$)!
- $\boldsymbol{x}_i$ is the value of $\boldsymbol{x}$ on the $i^{\text{th}}$ dimension

### Exercise 1

#### Question 1

Implement the Gaussian kernel $K$ in the following `gaussian_kernel()` Python function.

$$
K(\boldsymbol{u}, \boldsymbol{v})
= \exp\left(\frac{-||\boldsymbol{u} - \boldsymbol{v}||^2_2}{2 \sigma^2} \right)
$$

where $\sigma$ is a parameter equals to $1$ by default.

You can assume $u$ and $v$ to be simple scalars to simplify this Python implementation (i.e. restrict yourself to regression problem with 1 dimension inputs $x \in \mathbb{R}$).

Recall: $e^x$ is written `math.exp(x)` in Python.

In [ ]:
def gaussian_kernel(u, v, sigma = 1.):
    
    # TODO
    
    return None   # TODO

#### Question 2

Implement the Nadaraya-Watson kernel regression in the following `kernel_regression()` Python function.

$$
\text{kernel_regression}(\boldsymbol{x}, \mathcal{D})
= \frac{\sum^{n}_{i=1} K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) ~ y^{(i)}}{\sum^{n}_{i=1} K(\boldsymbol{x}^{(j)}, \boldsymbol{x})}
= y
$$

You can assume $x$ to be a simple scalar to simplify the Python implementation.
We assume `dataset` to contain examples $(\boldsymbol{x}^{(i)}, y^{(i)})$ of $\mathcal{D}$. Here, we assume `dataset` is a Pandas DataFrame having:
- one row per example
- a column "x" containing examples feature (only one dimension here)
- a column "y" containing examples label

**Hint**: you can use the following `for` loop to compute $\sum K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) ~ y^{(i)}$: `for xi, yi in zip(df.x, df.y)`.

In [ ]:
def kernel_regression(x, dataset):
    
    # TODO
    
    return None  # TODO

#### Question 3

We have the following dataset:

In [ ]:
dataset = pd.DataFrame([[2., 0.],
                        [5., 2.],
                        [7., 1.],
                        [10., 2.],
                        [14., 4.],
                        [16., 3.],
                        [17., 0.]], columns=['x', 'y'])
dataset

Check your `kernel_regression()` function with the following code:

In [ ]:
x_pred = np.linspace(0., 20., 200)
y_pred = [kernel_regression(x, dataset) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', label="Dataset", figsize=(12,8))
ax.plot(x_pred, y_pred, "-r", label="Kernel regression")
plt.legend();

### Correction

#### Question 1

In [ ]:
def gaussian_kernel(u, v, sigma = 1.):
    square_dist = (float(u) - float(v))**2
    return math.exp( -square_dist / (2. * sigma**2) )

Additional check of $K(0, x)$ for $x \in [-5, 5]$:

In [ ]:
x_check = np.linspace(-5., 5., 100)
y_check = [gaussian_kernel(0, x) for x in x_check]

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(x_check, y_check);

#### Question 2

In [ ]:
def kernel_regression(x, dataset):
    df = dataset
    w1 = sum([gaussian_kernel(xi, x) * yi for xi, yi in zip(df.x, df.y)])
    w2 = sum([gaussian_kernel(xj, x) for xj in df.x])
    return w1 / w2

#### Question 3

In [ ]:
x_pred = np.linspace(0., 20., 200)
y_pred = [kernel_regression(x, dataset) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', label="Dataset", figsize=(12,8))
ax.plot(x_pred, y_pred, "-r", label="Kernel regression")
plt.legend();

## *Weighted Least Squares*

For some regression problems, it may be helpful to give different importance to examples in the *learning set* $\mathcal{D} = \{(y^{(i)}, \boldsymbol{x^{(i)}})\}_{1 \leq i \leq n}$ that is to say associate a weight $\omega^{(i)}$ to example $\boldsymbol{x}^{(i)}$ in order to prioritize some of them and ignore some others (e.g. outliers).

Introducing these weights in the method of Least Square, the regression problem becomes:

$$E(\boldsymbol{\theta}) = \sum_{i=1}^n \omega^{(i)} (y^{(i)} - \boldsymbol{x}^{(i)} \boldsymbol{\theta})^2$$

In order to use the matrix notation, we put weights $\omega_i$ in the diagonal of the following matrix $\Omega$:

$$
\Omega =
\begin{pmatrix}
\omega^{(1)} & 0            & \cdots & 0 \\
0            & \omega^{(2)} & \cdots & 0 \\
\vdots       & \vdots       & \ddots & \vdots \\
0            & 0            & \cdots & \omega^{(n)} \\
\end{pmatrix}
$$

Then we can write:

$$E(\boldsymbol{\theta}) = (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta})^T \Omega (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta})$$

with:
$$
\boldsymbol{X} = \begin{pmatrix} 1 & x_1^{(1)} & \dots & x_p^{(1)} \\ \vdots & \vdots & \dots & \vdots \\ 1 & x_1^{(n)} & \dots & x_p^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{y} = \begin{pmatrix} y^{(1)} \\ \vdots \\ y^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{\theta} = \begin{pmatrix} \theta_0 \\ \vdots \\ \theta_p \end{pmatrix}
$$

### Exercise 2

#### Question 1

On a sheet of paper:
- Compute the analytic formulation of the gradient $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$
- Compute the analytic formulation of the optimal parameter $\boldsymbol{\theta^*}$

#### Question 2

Is it a convexe optimization problem like *Ordinary Least Squares* ?

#### Question 3

We have the following dataset and weights:

In [ ]:
X = np.array([[1, 1],
              [1, 2],
              [1, 3],
              [1, 4],
              [1, 5]])

y = np.array([1.8, 4.5, 3.4, 3.6, 4.2]).reshape([-1, 1])

Omega = np.diag([1, 2, 3, 2, 1])

In [ ]:
X

In [ ]:
y

In [ ]:
Omega

In [ ]:
plt.scatter(X[:,1], y);

Complete the following Python implementation of the `weighted_least_squares()` procedure.
It should return the optimal parameter $\boldsymbol{\theta^*}$ using the method of Least Square for the matrix of weights $\Omega$.

In [ ]:
def weighted_least_squares(X, Omega, y):
    
    # TODO

    theta = None  # TODO
    
    return theta

In [ ]:
theta = weighted_least_squares(X, Omega, y)
theta

Here, we expect $\theta$ to be a Numpy array of shape `(1, 2)` (i.e. a vector of two elements).

Numpy recall:
- The transpose of a matrix `X` is obtained with `X.T`
- The inverse of a matrix `X` is obtained with `np.linalg.inv(X)`
- The product of two matrices `X` and `Y` is obtained with `np.dot(X, Y)`
- The dot product of a matrix `X` and a vector `y` is obtained with `np.dot(X, y)`

#### Question 4

Check graphically your model using the following code:

In [ ]:
plot_regression_1d(X, y, theta, x_min=0, x_max=6)

#### Question 5

Change the weights in $\Omega$ to ignore the second point $x = 2$ (give the same weight to all other points) then recompute $\theta$ using `weighted_least_squares()` and check the results on plots with `plot_regression_1d()`.

### Correction

#### Question 1

C.f. lecture #3 slide 8

Recall:

$$
(\boldsymbol{A} + \boldsymbol{B})^T = \boldsymbol{A}^T + \boldsymbol{B}^T
$$

$$
(\boldsymbol{A} \boldsymbol{B})^T = \boldsymbol{B}^T \boldsymbol{A}^T
$$

\begin{align}
\text{Associativity:} & \quad
(\boldsymbol{A} \boldsymbol{B}) \boldsymbol{C}
= \boldsymbol{A} (\boldsymbol{B} \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} \boldsymbol{C}
\\
\text{Non-commutativity:} & \quad
\boldsymbol{A} \boldsymbol{B}
\neq \boldsymbol{B} \boldsymbol{A}
\\
\text{Distributivity:} & \quad
\boldsymbol{A} (\boldsymbol{B} + \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} + \boldsymbol{A} \boldsymbol{C} \\
& \quad
(\boldsymbol{B} + \boldsymbol{C}) \boldsymbol{A} 
= \boldsymbol{B} \boldsymbol{A} + \boldsymbol{C} \boldsymbol{A}
\end{align}

Differential identities for matrices:

\begin{align}
d(\boldsymbol{A}) & = 0  \quad \quad \quad \text{if } \boldsymbol{A} \text{ is not function of } \boldsymbol{X} \\
d(a\boldsymbol{X}) & = a d \boldsymbol{X}  \quad \quad \text{if } a \text{ is not function of } \boldsymbol{X} \\
d(\boldsymbol{X} +\boldsymbol{Y}) & = d \boldsymbol{X} + d \boldsymbol{Y} \\
d(\boldsymbol{XY}) & = (d \boldsymbol{X}) \boldsymbol{Y} + \boldsymbol{X} (d \boldsymbol{Y}) \\
d(\boldsymbol{X}^T) & = (d \boldsymbol{X})^T \\
\end{align}

Lets develop $E(\boldsymbol{\theta})$:

\begin{align}
E(\boldsymbol{\theta})
& = \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right)^T \boldsymbol{\Omega} \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) \\
& = \left( \boldsymbol{y}^T - (\boldsymbol{X} \boldsymbol{\theta})^T \right) \boldsymbol{\Omega} \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) \\
& = \left( \boldsymbol{y}^T - (\boldsymbol{\theta}^T \boldsymbol{X}^T) \right) \boldsymbol{\Omega} \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) \\
& = \left( \boldsymbol{y}^T \boldsymbol{\Omega} - \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \right) \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) \\
& = \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{y} - \underbrace{\boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta}}_{\text{scalar}} - \underbrace{\boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y}}_{\text{scalar}} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} \\
\end{align}

Since
$\boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta}$
is a $1 \times 1$ matrix, or a scalar, and its transpose
$(\boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta})^T = (\boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta})^T \boldsymbol{y} = (\boldsymbol{X} \boldsymbol{\theta})^T \boldsymbol{\Omega}^T \boldsymbol{y} = \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega}^T \boldsymbol{y} = \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y}$
is the same scalar ($\boldsymbol{\Omega} = \boldsymbol{\Omega}^T$ as $\boldsymbol{\Omega}$ is a diagonal matrix), then
$- \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} - \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y} = -2 \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} = -2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y}$.

Thus we have:

\begin{align}
E(\boldsymbol{\theta})
& = \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{y} -2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} \\
\end{align}

Then we can write $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$:

\begin{align}
\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})
& = \nabla_{\boldsymbol{\theta}} \left[ \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right)^T \boldsymbol{\Omega} \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) \right] \\
& = \nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{y} -2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} \right] \\
& = \nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{y}^T \boldsymbol{\Omega} \boldsymbol{y} \right] + \nabla_{\boldsymbol{\theta}} \left[-2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y} \right] + \underbrace{\nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} \right]}_{\text{see recall below}} \\
& = -2 \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y} + 2 \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{X} \boldsymbol{\theta} \\
& = -2 \boldsymbol{X}^T \boldsymbol{\Omega} (\boldsymbol{y} - \boldsymbol{X\theta})
\end{align}

Recall:

$$
\nabla_{\boldsymbol{\theta}} \left[\boldsymbol{\theta}^T \boldsymbol{A} \boldsymbol{\theta} \right] = 2 \boldsymbol{A} \boldsymbol{\theta} \quad \quad \text{if } \boldsymbol{A} \text{ is not a function of } \boldsymbol{\theta} \text{ and if } \boldsymbol{A} \text{ is a symmetric matrix}
$$

Then we can write the formula of the optimal parameter $\boldsymbol{\theta^*}$ (the one that minimize $E$ i.e. the one such that $\nabla_{\boldsymbol{\theta^*}} E(\boldsymbol{\theta^*}) = 0$):

\begin{align}
\nabla_{\boldsymbol{\theta^*}} E(\boldsymbol{\theta^*}) & = 0 \\
\Leftrightarrow -2 \boldsymbol{X}^T \boldsymbol{\Omega} (\boldsymbol{y} - \boldsymbol{X\theta^*}) & = 0\\
\Leftrightarrow \boldsymbol{X}^T \boldsymbol{\Omega} (\boldsymbol{y} - \boldsymbol{X\theta^*}) & = 0\\
\Leftrightarrow  \boldsymbol{X^T \Omega y} - \boldsymbol{X^T \Omega X\theta^*} & = 0\\
\Leftrightarrow \boldsymbol{X^T \Omega X\theta^*} & = \boldsymbol{X^T \Omega y} \\
\Leftrightarrow \boldsymbol{\theta^*} & = (\boldsymbol{X^T \Omega X})^{-1} \boldsymbol{X}^T \boldsymbol{\Omega} \boldsymbol{y}
\end{align}

#### Question 2

Yes, it's a convexe optimization problem like *Ordinary Least Squares*.

#### Question 3

In [ ]:
def weighted_least_squares(X, Omega, y):
    X_Omega_y = np.dot(np.dot(X.T, Omega), y)
    X_Omega_X = np.dot(np.dot(X.T, Omega), X)
    inv_X_Omega_X = np.linalg.inv(X_Omega_X)

    theta = np.dot(inv_X_Omega_X, X_Omega_y)
    
    return theta

In [ ]:
theta = weighted_least_squares(X, Omega, y)
theta

#### Question 4

In [ ]:
plot_regression_1d(X, y, theta, x_min=0, x_max=6)

#### Question 5

In [ ]:
Omega = np.diag([1, 0, 1, 1, 1])

theta = weighted_least_squares(X, Omega, y)

plot_regression_1d(X, y, theta, x_min=0, x_max=6)

Bonnus: let's check with another $\Omega$

In [ ]:
Omega = np.diag([0, 0, 1, 1, 0])

theta = weighted_least_squares(X, Omega, y)

plot_regression_1d(X, y, theta, x_min=0, x_max=6)

## *Local Linear Regression*

Another possible usage of *Weighted Least Squares* is the *Local Linear Regression*. It uses a *Kernel* $K(\boldsymbol{x}^{(i)}, \boldsymbol{x})$ to define the weight $\omega^{(i)}$ assigned to example $i$. Thus it's a linear regression giving more importance to examples close to the point $\boldsymbol{x}$ to predict. This mean that this method does a new fit (in other words it computes a new $\boldsymbol{\theta}^*$) for each new point to predict!

For each point $\boldsymbol{x}$ to predict:
1. Compute weights $\omega^{(i)}$ assigned to examples $\boldsymbol{x}^{(i)}$ w.r.t their distance to $\boldsymbol{x}$: $\omega^{(i)} = K(\boldsymbol{x}^{(i)}, \boldsymbol{x})$
2. Fit Weighted Least Squares to obtain the $\boldsymbol{\theta}^*$ vector associated to $\boldsymbol{x}$
3. Return the prediction $y = \boldsymbol{x\theta}^*$

### Exercise 3

We have the following dataset:

In [ ]:
dataset = gen_1d_polynomial_regression_samples(n_samples=30)

plot_1d_regression_samples(dataset)

#### Question 1

Complete the following Python implementation of the `locally_weighted_regression()` procedure defined above.
It should use the previously implemented `gaussian_kernel()` function (with `sigma=0.1`) and `weighted_least_squares()` function. It should return the predicted label $y$ corresponding to the input $\boldsymbol{x}$.

In [ ]:
def locally_weighted_regression(x, dataset, sigma=0.1):
    df = dataset   # Make an alias
    
    # Compute a weight wi for each example xi of the dataset: the closer xi is to x, the smaller wi is
    
    #wi = ...                                      # <- **TODO: UNCOMMENT AND COMPLETE**
    #Omega = ...                                   # <- **TODO: UNCOMMENT AND COMPLETE**
    
    # Fit weighted least squares to obtain theta ##
    
    intercept = np.ones(shape=len(df.x))
    X = np.array([intercept, df.x]).T
    y = df.y.values.reshape([-1, 1])
    #theta = weighted_least_squares(X, Omega, y)   # <- **TODO: UNCOMMENT**
    
    # Return prediction y = f(x) ##################
    
    # ...                                          # <- **TODO: UNCOMMENT AND COMPLETE**
    y = 0                                          # <- **TODO: UPDATE**
    
    return y

In [ ]:
x_pred = np.linspace(0., 1.5, 100)
y_pred = [locally_weighted_regression(x, dataset, sigma=0.1) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', figsize=(16, 8))
ax.plot(x_pred, y_pred);

#### Question 2

What happen when you change the value of the variable `sigma` parameter (try e.g. `sigma=0.3`)?
Why?

#### Question 3

Can we use Local Linear Regression to forecast time series as it was asked in the exercise 7 of the last tutorial? Why?

In [ ]:
try:
    URL = "natural_gas_co2_emissions_for_electric_power_sector.csv"
    df = pd.read_csv(URL, parse_dates=[0])
except Exception:
    URL = "https://raw.githubusercontent.com/jeremiedecock/polytechnique-cse204-2018/master/natural_gas_co2_emissions_for_electric_power_sector.csv"
    df = pd.read_csv(URL, parse_dates=[0])

df = pd.read_csv(URL, parse_dates=[0])

df['x'] = df.index
df['y'] = df.co2_emissions

df[['x','y']].head()

### Correction

#### Question 1

In [ ]:
dataset = gen_1d_polynomial_regression_samples(n_samples=30)

plot_1d_regression_samples(dataset)

In [ ]:
def locally_weighted_regression(x, dataset, sigma=0.1):
    df = dataset   # Make an alias
    
    # Compute a weight wi for each example xi of the dataset: the closer xi is to x, the smaller wi is
    wi = [gaussian_kernel(xi, x, sigma) for xi in df.x]
    Omega = np.diag(wi)
    
    # Fit weighted least squares to obtain theta
    intercept = np.ones(shape=len(df.x))
    X = np.array([intercept, df.x]).T
    y = df.y.values.reshape([-1, 1])
    theta = weighted_least_squares(X, Omega, y)
    
    # Return prediction y = f(x)
    y = theta[0] + theta[1] * x
    
    return y

In [ ]:
x_pred = np.linspace(0., 1.5, 100)
y_pred = [locally_weighted_regression(x, dataset, sigma=0.1) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', figsize=(16, 8))
ax.plot(x_pred, y_pred);

#### Question 2

In [ ]:
x_pred = np.linspace(0., 1.5, 100)
y_pred = [locally_weighted_regression(x, dataset, sigma=0.3) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', figsize=(16, 8))
ax.plot(x_pred, y_pred);

#### Question 3

This method has poor performance when the new point to predict is located outside the area known examples.

In [ ]:
try:
    URL = "natural_gas_co2_emissions_for_electric_power_sector.csv"
    df = pd.read_csv(URL, parse_dates=[0])
except Exception:
    URL = "https://raw.githubusercontent.com/jeremiedecock/polytechnique-cse204-2018/master/natural_gas_co2_emissions_for_electric_power_sector.csv"
    df = pd.read_csv(URL, parse_dates=[0])

df = pd.read_csv(URL, parse_dates=[0])

df['x'] = df.index
df['y'] = df.co2_emissions

df[['x','y']].head()

In [ ]:
x_pred = np.linspace(0., len(df) + 5, 1000)
y_pred = [locally_weighted_regression(x, df, sigma=0.5) for x in x_pred]

ax = df.plot.scatter(x='x', y='y', figsize=(16, 8))
ax.plot(x_pred, y_pred, "-r", label="Prediction")
plt.legend();

## Pathological cases

Consider the following implementation of the least squares method:

In [ ]:
def least_squares(X, y):
    XX = np.dot(X.T, X)
    Xy = np.dot(X.T, y)
    invXX = np.linalg.inv(XX)

    theta = np.dot(invXX, Xy)
    
    return theta

We want to use it to apply linear regularization to some datasets.

### Exercise 4

#### Question 1

What is wrong with the following dataset ?

In [ ]:
X = np.array([[1, 1.1, 2],
              [2, 2.2, 4]])
y = np.array([1.8, 2.7])

X

In [ ]:
#theta = least_squares(X, y)   # <- **TODO: UNCOMMENT**
#theta                         # <- **TODO: UNCOMMENT**

#### Question 2

What is wrong with the following dataset ?

In [ ]:
X = np.array([[1, 2, 3, 4, 5], [2, 4, 6, 8, 10]]).T
y = np.array([1.8, 2.7, 3.4, 3.8, 3.9])

In [ ]:
#theta = least_squares(X, y)   # <- **TODO: UNCOMMENT**
#theta                         # <- **TODO: UNCOMMENT**

### Correction

#### Question 1

Ordinary Least Squares requires to invert $\boldsymbol{X^TX}$.

But here we have $d > n$ i.e. we have greater input dimensionality $d$ than examples $n$. Thus there are no independent columns in $\boldsymbol{X}$ and the matrix $\boldsymbol{X^TX}$ is non-invertible.

#### Question 2

Here we have colinearity: the matrix $\boldsymbol{X^TX}$ is not full rank, determinant is zero and thus it's non-invertible.

In [ ]:
plt.scatter(X[:,0], X[:,1]);

Warning: here $\boldsymbol{x}$ has two dimensions here. Actually a 3D plot $(x_1, x_2, y)$ would show this colinearity more efficiently...

## Regularization with Ridge regression

We have the following dataset:

In [ ]:
x = np.array([1, 2.5, 3, 4.2, 5.5])
y = np.array([3.1, 3.5, 6.8, 10.9, 12.3])

plt.scatter(x, y);

We apply basis expansion to fit a polynomial model on the data.

In [ ]:
def basis_expansion(x, degree=5):
    Z_list = [np.ones(shape=x.shape)]   # intercept
    
    for deg_index in range(1, degree + 1):
        Z_list.append(x**deg_index)
    
    return np.array(Z_list).T

DEGREE = 7

Z = basis_expansion(x, degree=DEGREE)

# Make and fit the model

model = sklearn.linear_model.LinearRegression(fit_intercept=False)
model.fit(Z, y)

print("Coefs:", model.coef_)

# Compute the model's prediction

x_pred = np.linspace(x.min(), x.max(), 100)
Z_pred = basis_expansion(x_pred, degree=DEGREE)
Z_pred

y_pred = model.predict(Z_pred)

# Plot prediction and training set

fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(x_pred, y_pred)
ax.scatter(x, y)
plt.show();

As you can see, a polynomial function of degree 7 is certainly not adapter to fit efficiently these data. Here we have a clear over-fitting: the model is too complex for the data and it will have poor generalization performance (i.e. big error on new unknown data).

A solution is to reduce the complexity of the model using a lower polynomial degree.

An alternative is to apply a *regularization method* like the *ridge regularization* (a.k.a. *L2 regularization*) which applies a penalty on the value of $\theta$ to constrain it to be as small as possible.

A $\boldsymbol{\theta}$ with small elements usually make the model simpler and bring better generalization performances.

This L2 regularization is included in the least square method as follow:

$$
\boldsymbol{\theta}^*
\leftarrow \arg\min_{\boldsymbol{\theta}} E(\boldsymbol{\theta})
\quad \text{with} \quad
E(\boldsymbol{\theta})
= \underbrace{||\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta}||^2_2}_{\text{error term}} ~~ + \underbrace{\lambda ||\boldsymbol{\theta}||^2_2}_{\text{regularization}}$$

where $\lambda \in \mathbb{R}^+$ is the *regularization strength* coefficient:
- when $\lambda$ goes to infinity, the regularization term dominates the error term (MSE) and the coefficients $\boldsymbol{\theta}$ tend to zero
- when $\lambda$ goes to 0, the regularization term loose in importance and eventually the regularization term is ignored
- $\lambda$ is a *meta parameter*
- the best $\lambda$ for a problem can be computed empirically or automatically (e.g. grid search)

### Exercise 5

#### Question 1

On a sheet of paper:
- Compute the analytic formulation of the gradient $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$
- Compute the analytic formulation of the optimal parameter $\boldsymbol{\theta^*}$

#### Question 2

Is it a convexe optimization problem like *Ordinary Least Squares* ?

#### Question 3

Check the following Scikit Learn implementation of the Ridge Regression (more info here: https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression ).

In [ ]:
model = sklearn.linear_model.Ridge(alpha=6., fit_intercept=False)

model.fit(Z, y)

coefs = [model.intercept_] + model.coef_

print("Coefs:", coefs)

# Compute the model's prediction

x_pred = np.linspace(x.min(), x.max(), 100)
Z_pred = basis_expansion(x_pred, degree=DEGREE)
Z_pred

y_pred = model.predict(Z_pred)

# Plot prediction and training set

fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(x_pred, y_pred)
ax.scatter(x, y);
plt.show()

Change the value of the `alpha` parameter in `sklearn.linear_model.Ridge` and explain what happen (in Scikit Learn the $\lambda$ regression strength is named $\alpha$).

#### Question 4

Plot Ridge coefficients as a function of the regularization parameter.

Evaluate the following sequence of regularization strength: `alphas = np.logspace(-2, 5, 50)`.

#### Question 5

Update the following function to implement the ridge regression in Python (without using Scikit Learn). Check it as in question 3.

In [ ]:
def ridge_regression(X, y):
    XX = np.dot(X.T, X)
    Xy = np.dot(X.T, y)
    invXX = np.linalg.inv(XX)

    theta = np.dot(invXX, Xy)
    
    return theta

### Correction

#### Question 1

Recall:

$$
(\boldsymbol{A} + \boldsymbol{B})^T = \boldsymbol{A}^T + \boldsymbol{B}^T
$$

$$
(\boldsymbol{A} \boldsymbol{B})^T = \boldsymbol{B}^T \boldsymbol{A}^T
$$

\begin{align}
\text{Associativity:} & \quad
(\boldsymbol{A} \boldsymbol{B}) \boldsymbol{C}
= \boldsymbol{A} (\boldsymbol{B} \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} \boldsymbol{C}
\\
\text{Non-commutativity:} & \quad
\boldsymbol{A} \boldsymbol{B}
\neq \boldsymbol{B} \boldsymbol{A}
\\
\text{Distributivity:} & \quad
\boldsymbol{A} (\boldsymbol{B} + \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} + \boldsymbol{A} \boldsymbol{C} \\
& \quad
(\boldsymbol{B} + \boldsymbol{C}) \boldsymbol{A} 
= \boldsymbol{B} \boldsymbol{A} + \boldsymbol{C} \boldsymbol{A}
\end{align}

Differential identities for matrices:

\begin{align}
d(\boldsymbol{A}) & = 0  \quad \quad \quad \text{if } \boldsymbol{A} \text{ is not function of } \boldsymbol{X} \\
d(a\boldsymbol{X}) & = a d \boldsymbol{X}  \quad \quad \text{if } a \text{ is not function of } \boldsymbol{X} \\
d(\boldsymbol{X} +\boldsymbol{Y}) & = d \boldsymbol{X} + d \boldsymbol{Y} \\
d(\boldsymbol{XY}) & = (d \boldsymbol{X}) \boldsymbol{Y} + \boldsymbol{X} (d \boldsymbol{Y}) \\
d(\boldsymbol{X}^T) & = (d \boldsymbol{X})^T \\
\end{align}

Lets develop $E(\boldsymbol{\theta})$:

\begin{align}
E(\boldsymbol{\theta})
& = \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right)^T \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) + \lambda (\boldsymbol{\theta}^T \boldsymbol{\theta}) \\
& = \left( \boldsymbol{y}^T - (\boldsymbol{X} \boldsymbol{\theta})^T \right) \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \\
& = \left( \boldsymbol{y}^T - (\boldsymbol{\theta}^T \boldsymbol{X}^T) \right) \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right) + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \\
& = \boldsymbol{y}^T \boldsymbol{y} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta} - (\boldsymbol{\theta}^T \boldsymbol{X}^T) \boldsymbol{y} + (\boldsymbol{\theta}^T \boldsymbol{X}^T) (\boldsymbol{X} \boldsymbol{\theta}) + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \\
& = \boldsymbol{y}^T \boldsymbol{y} - \underbrace{\boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta}}_{\text{scalar}} - \underbrace{\boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y}}_{\text{scalar}} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \\
\end{align}

Since $\boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta}$ is a $1 \times 1$ matrix, or a scalar, and its transpose $(\boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta})^T = (\boldsymbol{X} \boldsymbol{\theta})^T \boldsymbol{y} = \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y}$ is the same scalar, then $- \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta} - \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y} = -2 \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\theta} = -2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y}$.

Thus we have:

\begin{align}
E(\boldsymbol{\theta})
& = \boldsymbol{y}^T \boldsymbol{y} \underbrace{-2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y}} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \\
\end{align}

Then we can write $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$:

\begin{align}
\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})
& = \nabla_{\boldsymbol{\theta}} \left[ \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right)^T \left( \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta} \right)  + \lambda (\boldsymbol{\theta}^T \boldsymbol{\theta}) \right] \\
& = \nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{y}^T \boldsymbol{y} -2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y} + \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta}  + \lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \right] \\
& = \nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{y}^T \boldsymbol{y} \right] + \nabla_{\boldsymbol{\theta}} \left[-2 \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{y} \right] + \underbrace{\nabla_{\boldsymbol{\theta}} \left[ \boldsymbol{\theta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} \right]}_{\text{see recall below}} + \nabla_{\boldsymbol{\theta}} \left[\lambda \boldsymbol{\theta}^T \boldsymbol{\theta} \right] \\
& = -2 \boldsymbol{X}^T \boldsymbol{y} + 2 \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} + 2\lambda \boldsymbol{I}_p \boldsymbol{\theta} \\
& = \left( \lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X} \right) 2\boldsymbol{\theta} - 2\boldsymbol{X}^T \boldsymbol{y}
\end{align}

where $\boldsymbol{I}_p \in \mathbb{R}^{p \times p}$ is the identity matrix of order $p$.

Recall:

$$
\frac{\partial \boldsymbol{\theta}^T \boldsymbol{A} \boldsymbol{\theta}}{\partial \boldsymbol{\theta}} = 2 \boldsymbol{A} \boldsymbol{\theta} \quad \quad \text{if } \boldsymbol{A} \text{ is not a function of } \boldsymbol{\theta} \text{ and if } \boldsymbol{A} \text{ is a symmetric matrix}
$$

Then we can write the formula of the optimal parameter $\boldsymbol{\theta^*}$ (the one that minimize $E(\boldsymbol{\theta})$ i.e. the one such that $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta}) = 0$):

\begin{align}
\nabla_{\boldsymbol{\theta^*}} E(\boldsymbol{\theta^*}) & = 0 \\
\Leftrightarrow \left( \lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X} \right) 2\boldsymbol{\theta^*} - 2\boldsymbol{X}^T \boldsymbol{y}
 & = 0\\
\Leftrightarrow \left( \lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X} \right) \boldsymbol{\theta^*} & = \boldsymbol{X}^T \boldsymbol{y}\\
\Leftrightarrow \boldsymbol{\theta^*} & = \left( \lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X} \right)^{-1} \boldsymbol{X}^T \boldsymbol{y}
\end{align}

As $\lambda > 0$, the matrix $\lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X}$ is always invertible.

#### Question 2

Yes, it's a convexe optimization problem like *Ordinary Least Squares*.

$E(\boldsymbol{\theta})$ is a quadratic form (convex function) thus it has a unique global minimum $\boldsymbol{\theta^*}$ where $\nabla_{\boldsymbol{\theta^*}} E(\boldsymbol{\theta^*}) = \boldsymbol{0}$

#### Question 3

In [ ]:
model = sklearn.linear_model.Ridge(alpha=6., fit_intercept=False)

model.fit(Z, y)

coefs = [model.intercept_] + model.coef_

print("Coefs:", coefs)

# Compute the model's prediction

x_pred = np.linspace(x.min(), x.max(), 100)
Z_pred = basis_expansion(x_pred, degree=DEGREE)
Z_pred

y_pred = model.predict(Z_pred)

# Plot prediction and training set

fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(x_pred, y_pred)
ax.scatter(x, y);
plt.show()

#### Question 4

In [ ]:
# Compute paths

alphas = np.logspace(-2, 5, 50)

coefs = []
for a in alphas:
    ridge = sklearn.linear_model.Ridge(alpha=a, fit_intercept=False)
    ridge.fit(Z, y)
    coefs.append(ridge.coef_)

# Display results

fig, ax = plt.subplots(figsize=(18, 8))

ax.plot(alphas, coefs)
ax.set_xscale('log')
#ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

#### Question 5

$$\boldsymbol{\theta^*} = \left( \lambda \boldsymbol{I}_p + \boldsymbol{X}^T \boldsymbol{X} \right)^{-1} \boldsymbol{X}^T \boldsymbol{y}$$

In [ ]:
def ridge_regression(X, y, lambda_):
    XX = np.dot(X.T, X)
    I = lambda_ * np.eye(XX.shape[0])
    Xy = np.dot(X.T, y)
    invXX = np.linalg.inv(I + XX)

    theta = np.dot(invXX, Xy)
    
    return theta

In [ ]:
theta = ridge_regression(Z, y, lambda_=6.)

print("Coefs:", theta)

# Compute the model's prediction

x_pred = np.linspace(x.min(), x.max(), 100)
Z_pred = basis_expansion(x_pred, degree=DEGREE)

y_pred = np.dot(Z_pred, theta)

# Plot prediction and training set

fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(x_pred, y_pred)
ax.scatter(x, y);
plt.show()